In [1]:
pip install trueskill

  Preparing metadata (setup.py) ... done
  Created wheel for trueskill: filename=trueskill-0.4.5-py3-none-any.whl size=18049 sha256=c3feb24633ce2823832d7a7c60e41a6e281272db3aedd1221d043522cd4e01d2
  Stored in directory: /root/.cache/pip/wheels/b9/4f/29/c79f0a2956775524c7a23638ac2b6fbb516c680f8e5eed9b53
Successfully built trueskill


In [5]:
import trueskill
import pandas as pd
import mpmath

# Set the backend to mpmath to handle floating point precision issues and configure TrueSkill parameters
env = trueskill.TrueSkill(backend='mpmath', draw_probability=0, beta=1, tau=0.05)

# Load and preprocess dataset
data = pd.read_csv("/content/og data set.csv")
relevant_columns = ['Name', 'Position', 'Club', 'Appearances', 'Wins', 'Losses', 'Goals', 'Assists', 'Tackles', 'Passes']
processed_data = data[relevant_columns].dropna()

# Function to get player's rating based on their statistics
def get_player_rating(player):
    # A basic skill score based on weighted stats (customizable)
    stats = (player['Appearances'] * 0.3 + player['Wins'] * 0.5 + player['Goals'] * 1.5 +
             player['Assists'] * 1.0 + player['Tackles'] * 0.2 + player['Passes'] * 0.1)
    return env.create_rating(stats)

# Function to retrieve player stats for the team
def get_team_stats(team_players):
    team_stats = []
    for player_name, position in team_players:
        player_data = processed_data[(processed_data['Name'] == player_name) & (processed_data['Position'] == position)]
        if not player_data.empty:
            team_stats.append(player_data.iloc[0])
    return pd.DataFrame(team_stats) if team_stats else pd.DataFrame(columns=relevant_columns)

# Function to compute team rating based on player names and positions
def get_team_ratings(team_players):
    ratings = []
    for player_name, position in team_players:
        player_data = processed_data[(processed_data['Name'] == player_name) & (processed_data['Position'] == position)]
        if not player_data.empty:
            player_rating = get_player_rating(player_data.iloc[0])
            ratings.append(player_rating)
    return ratings if ratings else [env.create_rating(0)]  # Return default rating if no players match

# Function to predict the winning team
def predict_winning_team(team1, team2):
    team1_ratings = get_team_ratings(team1)
    team2_ratings = get_team_ratings(team2)

    # Rate both teams using TrueSkill
    new_team1_ratings, new_team2_ratings = env.rate([team1_ratings, team2_ratings])

    # Calculate win probability
    delta_mu = sum(r.mu for r in new_team1_ratings) - sum(r.mu for r in new_team2_ratings)
    sum_sigma = sum(r.sigma ** 2 for r in new_team1_ratings) + sum(r.sigma ** 2 for r in new_team2_ratings)
    size = len(new_team1_ratings) + len(new_team2_ratings)
    denom = (size * (trueskill.BETA ** 2) + sum_sigma) ** 0.5
    win_probability = env.cdf(delta_mu / denom)

    return "Team 1 is more likely to win" if win_probability > 0.5 else "Team 2 is more likely to win"

# Function to take user input for a team
def get_user_team_input(team_number):
    team = []
    num_players = int(input(f"Enter the number of players for Team {team_number}: "))
    for _ in range(num_players):
        player_name = input("Enter player name: ")
        position = input("Enter player position: ")
        team.append((player_name, position))
    return team

# User input for both teams
print("Enter details for Team 1")
team1_players = get_user_team_input(1)

print("Enter details for Team 2")
team2_players = get_user_team_input(2)

# Get and display team stats
team1_stats = get_team_stats(team1_players)
team2_stats = get_team_stats(team2_players)

# Show the stats of both teams side by side
if not team1_stats.empty and not team2_stats.empty:
    print("\nTeam 1 Stats:")
    print(team1_stats.to_string(index=False))

    print("\nTeam 2 Stats:")
    print(team2_stats.to_string(index=False))
else:
    print("\nOne or both teams have no matching players in the dataset.")

# Predict the winning team
result = predict_winning_team(team1_players, team2_players)
print(f"\nPrediction: {result}")


Enter details for Team 1
Enter the number of players for Team 1: 1
Enter player name: Jack Grealish
Enter player position: Midfielder
Enter details for Team 2
Enter the number of players for Team 2: 1
Enter player name: Dan Burn
Enter player position: Defender

Team 1 Stats:
         Name   Position        Club  Appearances  Wins  Losses  Goals  Assists  Tackles  Passes
Jack Grealish Midfielder Aston-Villa           71    17      42      9        7     45.0    2098

Team 2 Stats:
    Name Position                     Club  Appearances  Wins  Losses  Goals  Assists  Tackles  Passes
Dan Burn Defender Brighton-and-Hove-Albion           44    11      19      0        0     73.0    1792

Prediction: Team 1 is more likely to win


In [2]:
pip install trueskill mpmath pandas
